CONNECT TO DRIVE

In [69]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


IMPORT ALL THE RELEVANT LIBRARIES

In [80]:
import pandas as pd
import numpy as np

IMPORT ALL THE DATASET

In [81]:
crime = pd.read_csv('/content/drive/MyDrive/MINDX Python/Crime.csv')
uniDict = pd.read_csv('/content/drive/MyDrive/MINDX Python/Dictionary for University_info.csv')
uniInfo = pd.read_csv('/content/drive/MyDrive/MINDX Python/University_info.csv')
ranking = pd.read_csv('/content/drive/MyDrive/MINDX Python/metro_startup_ranking.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (7,9,1427,1542,1561,1575,1725,1726,1727,1728) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


FILTER THE DATASET BASED ON CONDITIONS

1. Những trường ở khu vực thành thị sẽ có locale trong dãy 11, 12, 13 ở bảng uniInfo
2. Ở một thành phố có xếp hạng 75th percentile hoặc cao hơn ở cột startup rank trong bảng metro_startup_ranking
3. Ở một thành phố có thấp hơn 50th percentile về tổng số vụ phạm tội (total number of crimes) nên phải tỉnh tổng số lượng phạm tội ở mỗi thành phố
4. Có chương trình học về lĩnh vực "Computer And Information Sciences And Support Services". Khi đó thì cột PCIP11 sẽ lớn hơn 0 vì cột này thể hiện số phần trăm degree award cho degree Computer And Information Sciences And Support Services

In [82]:
# Take the dictionary of state to convert from the state fips to the actual name of the states
state_fips = uniDict.iloc[27:85, 7:9]
state_fips_dict = dict([(int(i),x) for i, x in zip(state_fips.VALUE, state_fips.LABEL)])

# Take the university whose locales fall within the range 11, 12, 13 and the proportion of degrees awarded for 
# Computer and Information Sciences and Support Services. Besides, the relevant columns include UNITID, CITY, ST_FIPS and STABBR for merging all the tables
uni_in_urban = uniInfo[(uniInfo['LOCALE']==11) | (uniInfo['LOCALE']==12) | (uniInfo['LOCALE']==13)]
uni_study_program = uni_in_urban[uni_in_urban['PCIP11']>0]
uni_study_program = uni_study_program[['UNITID','CITY','st_fips', 'STABBR','INSTNM','LOCALE']]
uni_study_program['state'] = uni_study_program['st_fips'].map(state_fips_dict)


# Take only the area whose startup Rank 
startupRank_75th = np.percentile(ranking['Startup Rank'], 75)
startupRank = ranking[ranking['Startup Rank']>=startupRank_75th]
startupRank = startupRank.drop(['Metro Area Code', 'Metro Area Name'], axis=1)
startupRank['Metro Area States'] = startupRank['Metro Area States'].str.split("-")
startupRank = startupRank.explode('Metro Area States')

# In the crime 
# Remove the null city in the column city
null_index = crime[crime['City'].isnull()].index.tolist()
for index in null_index:
  crime.iloc[index, 11] = crime.iloc[index,0].split(',')[0]

# Rename PuetoRico tag to PR
crime.iloc[371:,10] = 'PR'
criminalList = ['ViolentCrime','Murder','Rape','Robbery','AggravatedAssault','PropertyCrime','Burglary','Theft','MotorVehicleTheft']
for cri in criminalList:
    crime[cri] = crime[cri].astype(str).str.replace(",","")
    crime[cri] = crime[cri].astype('float64')
crime['total_crime'] = crime.sum(axis=1, numeric_only=True)
crime_50th = np.percentile(crime['total_crime'], 50)
crime = crime[crime['total_crime']<crime_50th]
crime_state = crime[['State', 'City','total_crime']]

df = uni_study_program.merge(startupRank, left_on=['state', 'CITY'], right_on=['Metro Area States', 'Metro Area Main City'], how='inner')
df = df.merge(crime_state, left_on=['STABBR','CITY'], right_on=['State','City'], how='inner')
df.drop(['Metro Area Main City','Metro Area States','State','City'], axis=1, inplace=True)

In [83]:
print(df['INSTNM'])

0                                     Walden University
1                                       Academy College
2                               American Indian OIC Inc
3                                      Augsburg College
4            The Art Institutes International-Minnesota
5                   University of Minnesota-Twin Cities
6           Minneapolis Community and Technical College
7                        Dunwoody College of Technology
8                                    Capella University
9                          Globe University-Minneapolis
10                          University of Detroit Mercy
11                                    Marygrove College
12              Wayne County Community College District
13                               Wayne State University
14                Community College of Allegheny County
15                      The Art Institute of Pittsburgh
16                                      Bradford School
17                           Carnegie Mellon Uni

PHƯƠNG PHÁP LỌC TRƯỜNG

Có tổng cộng 40 trường trong danh sách được chọn và các yếu tố đều được xem trọng tương đương nhau nên dùng ranking theo percentage để xếp điểm cho từng yếu tố và cộng tổng điểm lại để xét từ trên xuống dưới

In [84]:
df['total_crime_rank'] = df['total_crime'].rank(pct=True, ascending=False)
df['startup_rank'] = df['Startup Rank'].rank(pct=True, ascending=False)
df['locale_rank'] = df['LOCALE'].rank(pct=True, ascending=False)
df['sum_total'] = df['total_crime_rank']+df['startup_rank']+df['locale_rank']

In [88]:
df.sort_values(by='sum_total')[['INSTNM','CITY','state']]

,INSTNM,CITY,state
1,Academy College,Minneapolis,Minnesota
0,Walden University,Minneapolis,Minnesota
2,American Indian OIC Inc,Minneapolis,Minnesota
3,Augsburg College,Minneapolis,Minnesota
4,The Art Institutes International-Minnesota,Minneapolis,Minnesota
5,University of Minnesota-Twin Cities,Minneapolis,Minnesota
6,Minneapolis Community and Technical College,Minneapolis,Minnesota
7,Dunwoody College of Technology,Minneapolis,Minnesota
8,Capella University,Minneapolis,Minnesota
9,Globe University-Minneapolis,Minneapolis,Minnesota
